In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
class layer:
    def __init__(self, inputs, in_size, out_size, activation_function=None):
#         self.W = tf.Variable(tf.zeros([in_size, out_size]))
        self.W = tf.Variable(tf.random_normal([in_size, out_size]))
#         self.b = tf.Variable(tf.zeros([1,out_size]))
        self.b = tf.Variable(tf.constant(0.1, shape=[1,out_size]))
        self.Wx_plus_b = tf.matmul(inputs, self.W) + self.b
        self.activation_function = activation_function
    def output(self):
        if self.activation_function == None:
            result = self.Wx_plus_b
        else :
            result = self.activation_function(self.Wx_plus_b)
        return result

In [4]:
batch_size = 256
g_dim = 128

In [ ]:
x_d = tf.placeholder(tf.float32, shape=[None, 784])
x_g = tf.placeholder(tf.float32, shape=[None, 128])